In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

#PDFs in BDT and sindec?
import os

# set env flags to catch BLAS used for scipy/numpy 
# to only use 1 cpu, n_cpus will be totally controlled by csky
if False:
    os.environ['MKL_NUM_THREADS'] = "1"
    os.environ['NUMEXPR_NUM_THREADS'] = "1"
    os.environ['OMP_NUM_THREADS'] = "1"
    os.environ['OPENBLAS_NUM_THREADS'] = "1"
    os.environ['VECLIB_MAXIMUM_THREADS'] = "1"

import matplotlib as mpl
mpl.rcParams['figure.facecolor'] = 'w'
mpl.rcParams['savefig.facecolor'] = 'w'
import matplotlib.pyplot as plt
from matplotlib import colors, cm
import csky as cy
from csky import cext
import numpy as np
import astropy
#from icecube import astro
import histlite as hl
import healpy
import healpy as hp
import socket
import pickle
import copy
healpy.disable_warnings()
plt.rc('figure', facecolor = 'w')
plt.rc('figure', dpi=100)

## Define Settings

In [ ]:
selection_version = 'version-001-p00'

host_name = socket.gethostname()

if 'cobalt' in host_name:
    print('Working on Cobalts')
    data_prefix = '/data/user/ssclafani/data/cscd/final'
    ana_dir = '/data/user/ssclafani/data/analyses/'
    plot_dir = '/data/user/mhuennefeld/data/analyses/DNNCascadeCodeReview/unblinding_checks/plots/unblinding/galactic_plane_checks'
    
else:
    raise ValueError('Unknown host:', host_name)

In [ ]:
for dir_path in [plot_dir]:
    if not os.path.exists(dir_path):
        print('Creating directory:', dir_path)
        os.makedirs(dir_path)

## Load Data

In [ ]:
repo = cy.selections.Repository()
specs = cy.selections.DNNCascadeDataSpecs.DNNC_10yr

In [ ]:
%%time

ana = cy.get_analysis(
    repo, selection_version, specs, 
    #gammas=np.r_[0.1:6.01:0.125],
)

In [ ]:
a = ana.anas[0]
a.sig

In [ ]:
a.bg_data

## Helpers

In [ ]:
from cycler import cycle
from copy import deepcopy

soft_colors = cy.plotting.soft_colors
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']


def get_bias_allt(tr, ntrials=200, n_sigs=np.r_[:101:10], quiet=False):
    trials = [
        (None if quiet else print(f'\r{n_sig:4d} ...', end='', flush=True))
        or
        tr.get_many_fits(ntrials, n_sig=n_sig, logging=False, seed=n_sig)
        for n_sig in n_sigs]
    if not quiet:
        print()
    for (n_sig, t) in zip(n_sigs, trials):
        t['ntrue'] = np.repeat(n_sig, len(t))
    allt = cy.utils.Arrays.concatenate(trials)
    return allt

def get_color_cycler():
    return cycle(colors)

def plot_ns_bias(ax, tr, allt, label=''):

    n_sigs = np.unique(allt.ntrue)
    dns = np.mean(np.diff(n_sigs))
    ns_bins = np.r_[n_sigs - 0.5*dns, n_sigs[-1] + 0.5*dns]
    expect_kw = dict(color='C0', ls='--', lw=1, zorder=-10)

    h = hl.hist((allt.ntrue, allt.ns), bins=(ns_bins, 100))
    hl.plot1d(ax, h.contain_project(1),errorbands=True, 
              drawstyle='default', label=label)
    lim = ns_bins[[0, -1]]
    ax.set_xlim(ax.set_ylim(lim))
    ax.plot(lim, lim, **expect_kw)
    ax.set_aspect('equal')

    ax.set_xlabel(r'$n_{inj}$')
    ax.set_ylabel(r'$n_s$')
    ax.grid()

def plot_gamma_bias(ax, tr, allt, label=''):

    n_sigs = np.unique(allt.ntrue)
    dns = np.mean(np.diff(n_sigs))
    ns_bins = np.r_[n_sigs - 0.5*dns, n_sigs[-1] + 0.5*dns]
    expect_kw = dict(color='C0', ls='--', lw=1, zorder=-10)
    expect_gamma = tr.sig_injs[0].flux[0].gamma

    h = hl.hist((allt.ntrue, allt.gamma), bins=(ns_bins, 100))
    hl.plot1d(ax, h.contain_project(1),errorbands=True, 
              drawstyle='default', label=label)
    lim = ns_bins[[0, -1]]
    ax.set_xlim(lim)
    ax.set_ylim(1, 4)
    ax.axhline(expect_gamma, **expect_kw)

    ax.set_xlabel(r'$n_{inj}$')
    ax.set_ylabel(r'$\gamma$')
    ax.grid()

def plot_bkg_trials(
            bg, fig=None, ax=None, 
            label='{} bg trials', 
            label_fit=r'$\chi^2[{:.2f}\mathrm{{dof}},\ \eta={:.3f}]$', 
            color=colors[0],
            density=False,
            bins=50,
        ):
    if ax is None:
        fig, ax = plt.subplots(figsize=(6, 4))
    
    if density:
        h = bg.get_hist(bins=bins).normalize()
    else:
        h = bg.get_hist(bins=bins)
    if label is not None:
        label = label.format(bg.n_total)
    hl.plot1d(ax, h, crosses=True, color=color, label=label)

    # compare with the chi2 fit:
    if hasattr(bg, 'pdf'):
        x = h.centers[0]
        norm = h.integrate().values
        if label_fit is not None:
            label_fit = label_fit.format(bg.ndof, bg.eta)
        if density:
            ax.semilogy(x, bg.pdf(x), lw=1, ls='--', label=label_fit, color=color)
        else:
            ax.semilogy(x, norm * bg.pdf(x), lw=1, ls='--', label=label_fit, color=color)

    ax.set_xlabel(r'TS')
    if density:
        ax.set_ylabel(r'Density')
    else:
        ax.set_ylabel(r'number of trials')
    ax.legend()
        
    return fig, ax

## Setup Analysis

In [ ]:
import sys
sys.path.insert(0, '../..')

import config as cg

cg.base_dir = '/data/user/mhuennefeld/data/analyses/unblinding_v1.0.0/'

In [ ]:
def get_gp_tr(template_str, cutoff=np.inf, gamma=None, cpus=20):
    cutoff_GeV = cutoff * 1e3
    gp_conf = cg.get_gp_conf(
        template_str=template_str, gamma=gamma, cutoff_GeV=cutoff_GeV, base_dir=cg.base_dir)
    tr = cy.get_trial_runner(gp_conf, ana=ana, mp_cpus=cpus)
    return tr

def get_template_tr(template, gamma=2.7, cutoff_tev=np.inf, cpus=20):
    cutoff_gev = cutoff_tev * 1000.
    gp_conf = {
        'template': template,
        'flux': cy.hyp.PowerLawFlux(gamma, energy_cutoff=cutoff_gev),
        'randomize': ['ra'],
        'fitter_args': dict(gamma=gamma),
        'sigsub': True,
        'update_bg': True,
        'fast_weight': False,
    }
    tr = cy.get_trial_runner(gp_conf, ana=ana, mp_cpus=cpus)
    return tr


#### Get TrialRunners

In [ ]:
tr_dict = {
    'pi0': get_gp_tr('pi0'),
    'kra5': get_gp_tr('kra5'),
    'kra50': get_gp_tr('kra50'),
}

#### Get bkg fits for each template

In [ ]:
bkg_file_dict = {
    'pi0': '{}/gp/trials/{}/{}/trials.dict'.format(cg.base_dir, 'DNNC', 'pi0'),
    'kra5': '{}/gp/trials/{}/{}/trials.dict'.format(cg.base_dir, 'DNNC', 'kra5'),
    'kra50': '{}/gp/trials/{}/{}/trials.dict'.format(cg.base_dir, 'DNNC', 'kra50'),
}
n_bkg_trials = 20000
seed = 1337

bkg_dict = {}
for key, tr in tr_dict.items():
    if key in bkg_file_dict:
        print('Loading background trials for template {}'.format(key))
        sig = np.load(bkg_file_dict[key], allow_pickle=True)
        bkg_dict[key] = sig['poisson']['nsig'][0.0]['ts']
    
    else:
        print('Running background trials for template {}'.format(key))
        bkg_dict[key] = tr.get_many_fits(
            n_trials=n_bkg_trials, seed=seed, mp_cpus=20)
        

#### Get Results for each template

In [ ]:
res_dict = {}
for key in tr_dict.keys():
    f_path = os.path.join(
        cg.base_dir, 
        'gp/results/{}/{}_unblinded.npy'.format(key, key), 
    )
    res_dict[key] = np.load(f_path)

#### Plot ts distribution

In [ ]:
for key, bg in bkg_dict.items():
    bg_tsd = cy.dists.TSD(bg)
    fig, ax = plot_bkg_trials(bg_tsd)
    ts = res_dict[key][0]
    ns = res_dict[key][1]
    ax.axvline(
        ts, color='0.8', ls='--', lw=2,
        label='TS: {:3.3f} | ns: {:3.1f}'.format(ts, ns), 
    )
    ts_5sig = bg_tsd.isf_nsigma(5)
    ax.axvline(
        ts_5sig, ls='--', lw=1,
        label='5-sigma TS: {:3.3f}'.format(ts_5sig), 
    )
    ax.set_title('Template: {}'.format(key))
    ax.set_yscale('log')
    ax.legend()
    fig.savefig('{}/ts_dist_{}.png'.format(plot_dir, key))

#### Compute Significance

In [ ]:
p_val_dict = {}
sigma_dict = {}
max_n = 10000000
for key, bg in bkg_dict.items():
    bg_tsd = cy.dists.TSD(bg[:max_n])
    p_val_dict[key] = bg_tsd.sf(bg[:max_n])
    sigma_dict[key] = bg_tsd.sf_nsigma(bg[:max_n])


#### Plot Trial Correlation

In [ ]:
mask = np.zeros_like(bkg_dict['pi0'][:max_n])
sigma_threshold = 0.5

for key, tr in sigma_dict.items():

    mask = np.logical_or(mask, sigma_dict[key] > sigma_threshold)
    

In [ ]:
import matplotlib as mpl

def plot_corr_ax(ax, key1, key2, mask=None, norm=None):
    
    if mask is None:
        mask = np.ones_like(sigma_dict[key1], dtype=bool)
        
    ax.hist2d(
        sigma_dict[key1][mask], sigma_dict[key2][mask],
        bins=bins, norm=norm, cmin=1,
    )
    ax.plot(
        (bins[0][0], bins[0][-1]), (bins[0][0], bins[0][-1]), 
        ls='--', color='0.7', lw=3,
    )
    ax.set_xlabel('P-value of {}'.format(key1))
    ax.set_ylabel('P-value of {}'.format(key2))

fig, axes = plt.subplots(3, 1, figsize=(9, 9))

bins = (np.linspace(0, 6), np.linspace(0, 6))

plot_corr_ax(axes[0], 'pi0', 'kra5', mask=mask)
plot_corr_ax(axes[1], 'kra5', 'kra50', mask=mask)
plot_corr_ax(axes[2], 'pi0', 'kra50', mask=mask)
fig.tight_layout()


In [ ]:
max_nsigma = np.max(
    np.array([sigma_dict[k] for k in sigma_dict.keys()]),
    axis=0,
)

fig, ax = plt.subplots()
ax.hist(max_nsigma, bins=np.linspace(0, 6, 1000))
ax.set_xlabel('Max n-sigma')
ax.set_yscale('log')

bg_max = cy.dists.TSD(max_nsigma)


In [ ]:
from scipy import stats


nsigma_chosen = 4.68
pval_chosen = stats.norm.sf(nsigma_chosen)
nsigma_corrected = bg_max.sf_nsigma(nsigma_chosen)

pval_corrected = bg_max.sf(nsigma_chosen)
print(nsigma_corrected, pval_corrected/pval_chosen)
print(stats.norm.isf(pval_chosen * 3))


In [ ]:
ss_results = np.load(
    os.path.join(
        cg.base_dir, 'skyscan/results/unblinded_skyscan.npy'),
    allow_pickle=True,
)[()]
ss_trial = ss_results['ss_trial']



In [ ]:
ss_results


In [ ]:
# https://www.nasa.gov/mission_pages/GLAST/news/gammaray_best.html
# https://www.nasa.gov/images/content/317870main_Fermi_3_month_labeled_new.jpg
fermi_sources = {
    # ra, dec
    'NGC 1275': (049.9506656698585, +41.5116983765094),
    '3C 454.3': (343.49061658, +16.14821142),
    '47 Tuc': (006.022329, -72.081444),
    '0FGL J1813.5-1248': (273.349033, -12.766842),
    '0FGL J0614.3-3330': (093.5431162, -33.4983656),
    'PKS 0727-115': (112.57963530917, -11.68683347528),
    'Vela': (128.5000, -45.8333),
    'Geminga': (098.475638, +17.770253),
    'Crab': (083.63308, +22.01450),
    'LSI +61 303': (040.1319341179735, +61.2293308716971),
    'PSR J1836+5925': (279.056921, +59.424936),
    'PKS 1502+106': (226.10408242258, +10.49422183753),
}

In [ ]:
import utils

def plot_skymap(
            skymap, outfile=None, figsize=(9, 6),
            vmin=None, vmax=None, label=None, norm=None,
            cmap=cy.plotting.skymap_cmap,
            **kwargs
        ):
    """Plot a skymap

    Parameters
    ----------
    skymap : array_like
        The skymap to plot.
    outfile : str, optional
        The output file path to which to plot if provided.
    vmin : float, optional
        The minimum value for the colorbar.
    vmax : float, optional
        The maximum value for the colorbar.
    figsize : tuple, optional
        The figure size to use.
    label : str, optional
        The label for the colorbar.

    Returns
    -------
    fig, ax
        The matplotlib figure and axis.
    """
    fig, ax = plt.subplots(
        subplot_kw=dict(projection='aitoff'), figsize=figsize)
    sp = cy.plotting.SkyPlotter(
        pc_kw=dict(cmap=cmap, vmin=vmin, vmax=vmax, norm=norm),
        **kwargs
    )
    mesh, cb = sp.plot_map(ax, skymap, n_ticks=2)
    kw = dict(color='.5', alpha=.5)
    sp.plot_gp(ax, lw=.5, **kw)
    sp.plot_gc(ax, **kw)
    ax.grid(**kw)
    cb.set_label(label)
    fig.tight_layout()
    if outfile is not None:
        fig.savefig(outfile)

    return fig, ax, sp


cat_dict = {}
for cat_str in ['pwn', 'snr', 'unid']:
    catalog_file = os.path.join(
        cg.catalog_dir, '{}_ESTES_12.pickle'.format(cat_str))
    cat_dict[cat_str] = np.load(catalog_file, allow_pickle=True)




#### Plot Gamma

In [ ]:
cmap = plt.cm.viridis  # define the colormap
# extract all colors from the .jet map
cmaplist = [cmap(i) for i in range(cmap.N)]
# force the first color entry to be grey
cmaplist[0] = (.5, .5, .5, 1.0)

# create the new map
cmap = mpl.colors.LinearSegmentedColormap.from_list(
    'Custom cmap', cmaplist, cmap.N)

# define the bins and normalize
bounds = np.linspace(0, 4, 17)
print('bounds', bounds)
norm = mpl.colors.BoundaryNorm(bounds, cmap.N)



fig, ax, sp = plot_skymap(ss_trial[3], cmap=cmap, norm=norm, label=r'$\gamma$')
#fig, ax, sp = plot_skymap(ss_trial[3], cmap='viridis', label=r'$\gamma$')

for cat_str, cat in cat_dict.items():
    
    theta = np.pi/2. - np.deg2rad(cat.dec_deg)
    phi = np.deg2rad(cat.ra_deg)
    x, y = sp.thetaphi_to_mpl(theta, phi)

    ax.scatter(x, y, marker='x', color='red', label=cat_str)


# plot Fermi Sources
for key, (ra_deg, dec_deg) in fermi_sources.items():
    theta = np.pi/2. - np.deg2rad(dec_deg)
    phi = np.deg2rad(ra_deg)
    x, y = sp.thetaphi_to_mpl(theta, phi)
    ax.scatter(x, y, marker='+', color='1.0')

#### Plot ns

In [ ]:
fig, ax, sp = plot_skymap(ss_trial[2], cmap='viridis', label='ns')

for cat_str, cat in cat_dict.items():
    
    theta = np.pi/2. - np.deg2rad(cat.dec_deg)
    phi = np.deg2rad(cat.ra_deg)
    x, y = sp.thetaphi_to_mpl(theta, phi)

    ax.scatter(x, y, marker='x', color='red', label=cat_str)


# plot Fermi Sources
for key, (ra_deg, dec_deg) in fermi_sources.items():
    theta = np.pi/2. - np.deg2rad(dec_deg)
    phi = np.deg2rad(ra_deg)
    x, y = sp.thetaphi_to_mpl(theta, phi)
    ax.scatter(x, y, marker='+', color='1.0')

#### Plot N-sigmas

In [ ]:
ss_nsigmas =  stats.norm.isf(10**-ss_trial[0])


cmap = plt.cm.RdBu_r  # define the colormap
# extract all colors from the .jet map
cmaplist = [cmap(i) for i in range(cmap.N)]
# force the first color entry to be grey
cmaplist[0] = (.5, .5, .5, 1.0)

# create the new map
cmap = mpl.colors.LinearSegmentedColormap.from_list(
    'Custom cmap', cmaplist, cmap.N)

# define the bins and normalize
bounds = np.linspace(-4, 4, 18)
print('bounds', bounds)
norm = mpl.colors.BoundaryNorm(bounds, cmap.N)


#fig, ax, sp = plot_skymap(ss_nsigmas, cmap=)
fig, ax, sp = plot_skymap(ss_nsigmas, cmap=cmap, norm=norm, label=r'$n\cdot \sigma$')

# plot catalog
for cat_str, cat in cat_dict.items():
    
    theta = np.pi/2. - np.deg2rad(cat.dec_deg)
    phi = np.deg2rad(cat.ra_deg)
    x, y = sp.thetaphi_to_mpl(theta, phi)

    ax.scatter(x, y, marker='x', color='k', label=cat_str)

# plot hottest spots
for res_str in ['ipix_max_north', 'ipix_max_south']:
    theta, phi = hp.pix2ang(128, ss_results[res_str])
    x, y = sp.thetaphi_to_mpl(theta, phi)
    ax.scatter(x, y, marker='x', color='0.8')
    
# plot Fermi Sources
for key, (ra_deg, dec_deg) in fermi_sources.items():
    theta = np.pi/2. - np.deg2rad(dec_deg)
    phi = np.deg2rad(ra_deg)
    x, y = sp.thetaphi_to_mpl(theta, phi)
    ax.scatter(x, y, marker='+', color='green')


fig.savefig('{}/skymap_ns_with_catalog.png'.format(plot_dir))

In [ ]:
for ipx in np.argsort(ss_nsigmas)[::-1][:5]:
    theta, phi = hp.pix2ang(128, ipx)
    dec = np.pi/2.  - theta
    print(np.rad2deg(dec), np.rad2deg(phi))

In [ ]:
np.argsort(ss_nsigmas_cpy)[::-1]

In [ ]:
ra_deg, dec_deg = fermi_sources['Crab']
theta = np.pi/2. - np.deg2rad(dec_deg)
phi = np.deg2rad(ra_deg)
mask_crab = hp.query_disc(
    nside=128, 
    vec=hp.dir2vec(theta=theta, phi=phi), 
    radius=np.deg2rad(10),
)
ipx_crab = hp.ang2pix(128, theta=theta, phi=phi)



ss_nsigmas_cpy = np.zeros_like(ss_nsigmas)
ss_nsigmas_cpy[mask_crab] = np.array(ss_nsigmas[ipix_crab])
fig, ax, sp = plot_skymap(ss_nsigmas_cpy, label=r'$n\cdot \sigma$')

for ipx in np.argsort(ss_nsigmas_cpy)[::-1][:5]:
    theta, phi = hp.pix2ang(128, ipx)
    dec = np.pi/2.  - theta
    print('{:3.3f} {:3.3f} | {:3.3f} $n\cdot \sigma$'.format(
        np.rad2deg(phi), np.rad2deg(dec), ss_nsigmas_cpy[ipx]))
print(fermi_sources['Crab'], ss_nsigmas[ipx_crab])